In [129]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [130]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [131]:
from git import Repo

In [132]:
from pathlib import Path

In [133]:
PATH_TO_BLOG_REPO = Path('/Users/peterhyland/Documents/GitHub/peter-hyland.github.io/.git')

In [134]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [135]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [136]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [137]:
def update_blog(commit_message='Updates blog'):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [138]:
update_blog()

In [139]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image = Path(cover_image)

    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title

    shutil.copy(cover_image,PATH_TO_CONTENT)

    if not os.path.exists(path_to_new_content): 
        with open(path_to_new_content,'w') as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")

            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br/>\n")
            f.write(f"<h1> {title} <h1>")
            f.write(content.replace("\n", "<br/>\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("blog created")
            return path_to_new_content
    else:
        raise FileExistsError("file already exists")

In [140]:
path_to_new_content = create_new_blog('Test_title', 'rbediewf','logo.png')

blog created


In [141]:
from bs4 import BeautifulSoup as Soup
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [142]:
str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n<title>Document</title>\n</head>\n<body>\n<h1>my blog home</h1>\n<a href="index.html">home</a>\n</body>\n</html>'

In [143]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

In [144]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    print(links)
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))

In [145]:
write_to_index(path_to_new_content)

[<a href="index.html">home</a>]


In [ ]:
update_blog()